This notebook will use the bathymetry rasters generated in 01a_get_ehydro.ipynb to get cloud-masked Sentinel-2 L2A imagery.

In [ ]:
import os
from osgeo import gdal
import rasterio
import ee
import geemap
import numpy as np
from pyproj import Transformer
from datetime import datetime, timedelta
import re

In [ ]:
ee.Initialize(project = '') ##enter your project name here as a string to initialize exchanges with ee api

# Functions

In [ ]:
# FUCNTIONS FROM https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless 
# USE FOR CLOUD MASKING

def get_s2_sr_cld_col(aoi, start_date, end_date, cloud_filter):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    combined_coll = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

    return combined_coll.map(lambda img: img.clip(aoi))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def export_image_to_drive(image, description, aoi):
    """
    Export a single image to Google Drive.

    Args:
        image: ee.Image, the image to be exported.
        description: str, unique description for the export task.
        aoi: ee.Geometry, the area of interest for the export.
    """

    image = image.select(['B2', 'B3', 'B4', 'B8'])
    # Setup the export task
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        region=aoi,  # Make sure the geometry (aoi) is defined earlier
        fileFormat='GeoTIFF',
        scale=10  # Adjust the scale as needed
    )
    task.start()
    print(f'Exporting {description} to Drive...')


In [ ]:
def extract_date(filepath):
    """extract search date window from the eHydro data"""
    match = re.search(r'\d{4}\d{2}\d{2}', filepath)
    date = datetime.strptime(match.group(), '%Y%m%d')
    return (date - timedelta(days=1)).strftime('%Y-%m-%d'), (date + timedelta(days=1)).strftime('%Y-%m-%d')

def extract_valid_bounds_to_epsg4326(raster_path):
    """Extracts the bounding box of valid (non-NaN) data from a raster and converts it to EPSG:4326."""
    with rasterio.open(raster_path) as src:
        # Read the raster data
        data = src.read(1)  # Assuming a single band
        transform = src.transform  # Affine transform of the raster
        src_crs = src.crs  # Source CRS of the raster

        # Create a mask for valid (non-NaN) pixels
        valid_mask = ~np.isnan(data)

        # Find the row and column indices of valid pixels
        valid_rows, valid_cols = np.where(valid_mask)

        if valid_rows.size == 0 or valid_cols.size == 0:
            raise ValueError("No valid data in the raster.")

        # Calculate the geographic coordinates of the valid bounds
        min_row, max_row = valid_rows.min(), valid_rows.max()
        min_col, max_col = valid_cols.min(), valid_cols.max()

        # Use the transform to convert row/col to geographic bounds
        min_x, min_y = rasterio.transform.xy(transform, min_row, min_col, offset="ul")
        max_x, max_y = rasterio.transform.xy(transform, max_row, max_col, offset="ul")

        # Bounds in the source CRS
        bounds_src_crs = (min_x, min_y, max_x, max_y)

        # Transform bounds to EPSG:4326
        transformer = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
        min_x_4326, min_y_4326 = transformer.transform(min_x, min_y)
        max_x_4326, max_y_4326 = transformer.transform(max_x, max_y)

        bounds_epsg4326 = (min_x_4326, min_y_4326, max_x_4326, max_y_4326)
        bbox = ee.Geometry.BBox(bounds_epsg4326[0], bounds_epsg4326[1], bounds_epsg4326[2], bounds_epsg4326[3])

    return bbox

# Establish working directories

In [ ]:
usace_code = 'CESWG'
BATHY_PATH = f'/home/clay/Documents/SDB/{usace_code}/bathy_rasters'        # STORAGE_DIR from 01a_get_ehydro.ipynb
S2_PATH = f'/home/clay/Documents/SDB/{usace_code}/bathy'
os.makedirs(S2_PATH, exist_ok=True)

In [ ]:
bathy_rasters = [os.path.join(BATHY_PATH, f) for f in os.listdir(BATHY_PATH) if f.endswith('.tif')]

# Extract the bounds and dates needed for Sentinel-2 that correspond with each survey

In [ ]:
# i think this only extracts bbox of the entire raster, will try to get just the valid pixels

valid_bounds = extract_valid_bounds_to_epsg4326(bathy_rasters[0])   # used to get bbox in epsg:4326 as a gee geometry for searching
searchdates = extract_date(bathy_rasters[0])                        # gets the dat before and after the eHydro survey for retrieving Sentinel-2 image

In [ ]:
geeinfo = {}
for raster in bathy_rasters:
    
    date = extract_date(raster)
    bounds = extract_valid_bounds_to_epsg4326(raster)

    geeinfo[raster] = [bounds, date]
surveykeys = list(geeinfo.keys())

# Get Sentinel-2 images

In [ ]:
s2_cloud_cov = 20

for survey, items in geeinfo.items():
    aoi = items[0]
    dates = items[1]

    coll = get_s2_sr_cld_col(aoi, dates[0], dates[1], s2_cloud_cov)

    if coll.size().getInfo() > 0:
        geeinfo[survey].append(coll)

In [ ]:
# cloud-masking cell, will preserve the non-cloudy pixels for the most amount of training pixels

CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100
for survey in goodsurveys:
    s2_sr_median = (geeinfo[survey][2].map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
    geeinfo[survey].append(s2_sr_median)

# Export images

In [ ]:
for survey in goodsurveys:
    export_image_to_drive(geeinfo[survey][-1], survey, geeinfo[survey][0])

print('='* 250)
print('Finished uploading to Drive.')